In [1]:
import sys
import os
sys.path.append("/home/pi/EE123/Chirp-EE123")

cur_path = os.path.dirname('/home/pi/EE123/Chirp-EE123')
sys.path.insert(0, '/home/pi/EE123/Chirp-EE123/src')

from utils.EE123TNC import *
from utils.hamming import *
from compression.compressData import *

#from compression.pca import *

import sounddevice as sd
import RPi.GPIO as GPIO

Loading BokehJS ...

# Compress image/gif

In [41]:
compressT = CompressData("milkyway.png")
compressT.compress(params= {'factor_xy': 0.054, 'timeFlag': False, 'frame_rate':10})
ori_Data = compressT.image_stack
com_size = compressT.com_video_size
ori_size = np.prod(ori_Data.shape)
print(com_size)
print(ori_size)

compressR = CompressData("milkyway.png")
result, frame_rate = compressR.decompress()
npArray_play(result, frame_rate = frame_rate)
#PSNR = psnr(ori_Data, result)
#print(PSNR)

5616
2182950


# Load image/gif into queue

In [42]:
fname = b"../../asset/buffer.txt"
f = open(fname,"rb")

modem = TNCaprs(fs = 48000, Abuffer = 512, Nchunks = 10)
print("Putting packets in Queue")

starttime = time.time()
npack = 0
num_bits_per_packet = 256
prefix = bitarray(np.tile([0,1,1,1,1,1,1,0],(5,)).tolist())
Qout=Queue.Queue()

while True:
    chunk = f.read(num_bits_per_packet)
    if chunk:
        info = bytes(chunk)
        info += bytes(256 - len(info))
        packet = ax25.UI(info=info)
        bits=modem.NRZ2NRZI(prefix+packet.unparse()+prefix)
        packets = modem.modulate(bits)
        Qout.put(packets)
    else:
        break

print("Processing took:", time.time()-starttime)
print("Done")

f.close()

Putting packets in Queue
Processing took: 9.65214204788208
Done


# Transmit image/gif

In [43]:
GPIO.cleanup()
GPIO.setmode(GPIO.BOARD)
PTT = 16
GPIO.setup(PTT, GPIO.OUT, initial = 0)

print("Starting transmission...")
starttime = time.time()

sig = np.array([])

GPIO.output(PTT, GPIO.HIGH)
while(not(Qout.empty())):
    sig = np.append(sig, Qout.get())
    
sd.play(sig*0.2,samplerate=48000,device=usb_idx, blocking=True)
GPIO.output(PTT, GPIO.LOW)

print("Done")
print(time.time() - starttime)

Starting transmission...
Done
75.86220264434814
